# Red Team KubeHound Workflow

A step by step example workflow of how to use KubeHound for Red Team operations.

## Initial Setup

Connection is being initated directly from the docker using the env vars `GRAPH_NOTEBOOK_HOST` and `GRAPH_NOTEBOOK_PORT`. To overwrite it you can use the magic `%%graph_notebook_config` [details here](https://github.com/aws/graph-notebook/tree/main/additional-databases/gremlin-server#connecting-to-a-local-gremlin-server-from-jupyter).

Now set the appearance customizations for the notebook. You can see a guide on possible options [here](https://github.com/aws/graph-notebook/blob/623d43827f798c33125219e8f45ad1b6e5b29513/src/graph_notebook/notebooks/01-Neptune-Database/02-Visualization/Grouping-and-Appearance-Customization-Gremlin.ipynb#L680)

In [ ]:
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

## Worfklow

### Initial Recon

Look for exposed endpoints attached to containers and return the port details and image. This enables us to match against any exploits in our catalogue

In [ ]:
%%gremlin
kh.endpoints().
	where(out().hasLabel("Container")).
	project('port',"portName", 'image').
	by(values("port")).
	by(values("portName")).
	by(out().hasLabel("Container").values("image")).
	dedup()

Now let's focus on those with a critical attack path

In [ ]:
%%gremlin
kh.endpoints().
	hasCriticalPath().
	where(out().hasLabel("Container")).
	project('port',"portName", 'image').
	by(values("port")).
	by(values("portName")).
	by(out().hasLabel("Container").values("image")).
	dedup()

### Attack Path Analysis

Let's pick a promising target we have an exploit for and look at possible attack paths

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.endpoints().
	has("portName", "elasticsearch").	// Change the value here for those found
	not(has("protocol", "UDP")). // Exclude or change based on requirements
	criticalPaths().
	by(elementMap()).
	limit(100)	// Limit the number of results for large clusters

Now to refine our analysis, let's discount a few of the noisier attacks and focus on shorter attack paths. See [reference documentation](https://kubehound.io/reference/attacks/) for detaills of these attacks.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.endpoints().
	has("portName", "elasticsearch").  // Change the value here for those found
	criticalPathsFilter(6, "TOKEN_BRUTEFORCE", "POD_EXEC", "POD_CREATE").
	by(elementMap()).
    limit(100)	// Limit the number of results for large clusters

Now let's explore a couple of interesting endpoints which do not have a critical path BUT get us to a Node which could contain useful resources outside the scope of KubeHound e.g AWS credentials, SSH keys, etc

In [ ]:
%%gremlin -d name -g class -le 50 -p inv,oute
kh.endpoints().
	has("portName", within("jmx", "ssh", "log4j")). // Change the values here for those found
	repeat(
		outE().inV().
		simplePath()).
	until(
		hasLabel("Node").
		or().
		loops().is(5)).
	hasLabel("Node").
	path().
	by(elementMap()).
    limit(100)	// Limit the number of results for large clusters